In [1]:
using LinearAlgebra

In [2]:
A = [1 2 3; 4 5 6; 7 8 9]

3×3 Matrix{Int64}:
 1  2  3
 4  5  6
 7  8  9

In [3]:
A[2,1]

4

In [4]:
Base.setindex!(A, 12, 3)

3×3 Matrix{Int64}:
  1  2  3
  4  5  6
 12  8  9

In [5]:
Base.size(A)

(3, 3)

In [19]:
struct PasovnaMatrika{T} <: AbstractArray{T, 2}
    pasovi::Dict{Int, Vector{T}} # 0 diagonala, 1, 2, 3...nad diagonalo, -1, -2, ... pod diagonalo
    n::Int
    # function PasovnaMatrika{T}(pasovi::Dict{Int, Vector{T}}, n::Int) where T
    #     new(pasovi, n)
    # end
end

In [283]:
struct ZgornjePasovnaMatrika{T} <: AbstractArray{T, 2}
    pasovi::Dict{Int, Vector{T}} # 0 diagonala, 1, 2, 3...nad diagonalo, -1, -2, ... pod diagonalo
    n::Int
    # function PasovnaMatrika{T}(pasovi::Dict{Int, Vector{T}}, n::Int) where T
    #     new(pasovi, n)
    # end
end

In [284]:
struct SpodnjePasovnaMatrika{T} <: AbstractArray{T, 2}
    pasovi::Dict{Int, Vector{T}} # 0 diagonala, 1, 2, 3...nad diagonalo, -1, -2, ... pod diagonalo
    n::Int
    # function PasovnaMatrika{T}(pasovi::Dict{Int, Vector{T}}, n::Int) where T
    #     new(pasovi, n)
    # end
end

In [7]:
function Base.size(M::PasovnaMatrika)
    return (M.n, M.n)
end

In [8]:
function Base.getindex(M::PasovnaMatrika, I::Vararg{Int, 2})
    pas = I[2] - I[1]
    if haskey(M.pasovi, pas)
        if pas >= 0
            return M.pasovi[pas][I[1]]
        else 
            return M.pasovi[pas][I[2]]
        end
    else
        return 0
    end
end

In [9]:
function Base.setindex!(M::PasovnaMatrika, v, I::Vararg{Int, 2})
    pas = I[2] - I[1]
    if haskey(M.pasovi, pas)
        if pas >= 0
            M.pasovi[pas][I[1]] = v
        else 
            M.pasovi[pas][I[2]] = v
        end
    else
        return error("Vrednosti se lahko nastavi samo v neničelnih pasovih") # dodaj cel pas naenkrat? -> setindex!(A, X, I...)
    end
end

In [10]:
import Base.*, Base.\

In [71]:
[zeros(2); 3; zeros(3)]

6-element Vector{Float64}:
 0.0
 0.0
 3.0
 0.0
 0.0
 0.0

In [108]:
function *(M::PasovnaMatrika, v::Vector)
    y = zeros(length(v))
    for (k, pas) in M.pasovi
        # print(max(-k, 0), "  ", min(k, 0), " ", k, " ", "\n")
        # curr = [zeros(max(k, 0)); pas; zeros(max(-k, 0))]
        # print(curr," cc ", curr .* v, "\n")
        
        for i=1:length(pas)
            row = k<=0 ? -k+i : k+i # indeks elemneta v pasu
            col = k<=0 ? i : k+i # indeks elemnta v pasu
            y[row] += v[col] * pas[i]
        end
        # y += curr .* v
        # if k > 0
        #     #add zeros to the end
        #     curr = zeros(pas)
        #     curr = vcat(pas, curr)
        #     y += curr .* v
        # elseif k < 0
        #     curr = zeros(abs(pas))
        #     curr = vcat(curr, pas)
        #     y += curr .* v
        # else
        #     y += pas .* v
        # end
    end
    return y
end

[1.0, -21.5, -2.8139534883720927] cc [1.0, -21.5, -2.8139534883720927]
[0.0, 0.0, 0.0] cc [0.0, 0.0, 0.0]
[0.0, 0.0, 8.0] cc [0.0, 0.0, 8.0]
[0.0, 6.0, -25.0] cc [0.0, 6.0, -25.0]


* (generic function with 365 methods)

In [242]:
function \(M::PasovnaMatrika, b::Vector)
    # eliminacija  če je zg/sp trikotna ni treba tega
    d = M.pasovi[0]
    # for (k, pas) in M.pasovi
    bc = copy(b)
    
    for k=-M.n:-1
        if haskey(M.pasovi, k)
            pas = M.pasovi[k] # najbolj spodnji pas

            # prvega uničimo od zgoraj naslednje pa od zgoraj z updatane matrike (kjer so spredaj 0)
            for i=1:length(pas)
                l = pas[i] / M[-k+i-1, i]
                print(l, " l, \n")
                # dv = copy(M[-k+i-1, :])
                # odštejemo po vrstici
                for ii=i:M.n # tukaj bi se lahko že prej končalo - ko smo na zadnji diagonali v vrstici i, mogoče vključi št zg in sp diagonal
                    print(-k+i, " ii ", ii, " i ", i, " mm ", M[-k+i-1, ii], "\n")
                    M[-k+i, ii] -= M[-k+i-1, ii] * l
                    
                    print(M[-k+i, ii], " res \n")
                end
                bc[-k+i] -= bc[-k+i-1] * l
                print("--------\n")
            # še prostale elemnte vrstice damo na 0


            end
        end
    end
    # vstavljanje
    print(M, "\n")
    print(bc, "\n")
    x = zeros(M.n)
    for i=M.n:-1:1
        curr = bc[i] / d[i]
        print(curr, "\n")
        for ii=M.n:-1:i
            curr -= M[i, ii] * x[ii]
        end
        x[i] = curr
    end
    return x
end



\ (generic function with 136 methods)

In [324]:
import LinearAlgebra.lu

function lu(M::PasovnaMatrika)
    # eliminacija  če je zg/sp trikotna ni treba tega

    # for (k, pas) in M.pasovi
    L = Matrix{Float64}(I,M.n, M.n)
    l_pas = minimum(M.pasovi.keys)
    z_pas = maximum(M.pasovi.keys)
    print(l_pas, "bbb", z_pas, "\n")
    A = deepcopy(M)
    
    for j=1:M.n
        for i=j+1:-l_pas+1
            # pas = M.pasovi[k] # najbolj spodnji pas
            l = A[i, j] / A[j,j]
            L[i, j] = l

            for k=j+1:min(z_pas+j+1, M.n)
                if A[j,j] < A[j,k]
                    error("Ni diagonalno dominanbsata")
                end
                print(i, " ", k, "\n")
                A[i, k] = A[i, k] - l * A[j, k]
            end
        end
    end 

    #   ŠE U ZGORNJE/SPODNJE TRIKOTNO PRETVORI
    print(A, "\n")
    display(L)

    for i=1:M.n 
        for j=1:M.n 
            if i>j
                A[i, j] = 0
            end
        end
    end

    return (L, A)

end


lu (generic function with 12 methods)

In [325]:
P = PasovnaMatrika(Dict([(0,[10.0, 15.0, 9.0]), (-1,[4.0, 8.0]), (1, [2.0, 6.0]), (-2, [7.0])]), 3)
display(P)
L, U = lu(P)

3×3 PasovnaMatrika{Float64}:
 10.0   2.0  0
  4.0  15.0  6.0
  7.0   8.0  9.0

3×3 Matrix{Float64}:
 1.0  0.0       0.0
 0.4  1.0       0.0
 0.7  0.464789  1.0

-2bbb140525160028896
2 2
2 3
3 2
3 3
3 3
[10.0 2.0 0; 4.0 14.2 6.0; 7.0 6.6 6.211267605633803]


([1.0 0.0 0.0; 0.4 1.0 0.0; 0.7 0.4647887323943662 1.0], [10.0 2.0 0; 0.0 14.2 6.0; 0.0 0.0 6.211267605633803])

In [326]:
L*U

3×3 Matrix{Float64}:
 10.0   2.0  0.0
  4.0  15.0  6.0
  7.0   8.0  9.0

In [307]:
P = PasovnaMatrika(Dict([(0,[1.0, 5.0, 9.0]), (-1,[4.0, 9.2]), (1, [2.0, 6.0]), (-2, [7.0])]), 3)
L, U = lu(P)

ErrorException: Ni diagonalno dominanbsata

In [304]:
U

3×3 PasovnaMatrika{Float64}:
 1.0   2.0   0
 0.0  -3.0   6.0
 0.0   0.0  -2.04

In [305]:
L*U


3×3 Matrix{Float64}:
 1.0  2.0   0.0
 4.0  5.0   6.0
 7.0  8.48  9.0

In [243]:
P = PasovnaMatrika(Dict([(0,[1.0, 2.5, 3.0]), (-1,[4.0, 5.0]), (1, [6.0, 7.0]), (2, [8.0])]), 3)
v = [3.0, 2, 1]

P

3×3 PasovnaMatrika{Float64}:
 1.0  6.0  8.0
 4.0  2.5  7.0
 0    5.0  3.0

In [244]:
P = PasovnaMatrika(Dict([(0,[1.0, 5.0, 9.0]), (-1,[4.0, 8.0]), (1, [2.0, 6.0]), (-2, [7.0])]), 3)
v = [1.0, 2, 3]

3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [245]:
A = [1 2 0; 4 5 6; 7 8 9]
lu(A)

LU{Float64, Matrix{Float64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0  0.0
 0.142857  1.0  0.0
 0.571429  0.5  1.0
U factor:
3×3 Matrix{Float64}:
 7.0  8.0        9.0
 0.0  0.857143  -1.28571
 0.0  0.0        1.5

In [246]:
x = P\v

1.75 l, 
3 ii 1 i 1 mm 4.0
0.0 res 
3 ii 2 i 1 mm 5.0
-0.75 res 
3 ii 3 i 1 mm 6.0
-1.5 res 
--------
4.0 l, 
2 ii 1 i 1 mm 1.0
0.0 res 
2 ii 2 i 1 mm 2.0
-3.0 res 
2 ii 3 i 1 mm 0
6.0 res 
--------
0.25 l, 
3 ii 2 i 2 mm -3.0
0.0 res 
3 ii 3 i 2 mm 6.0
-3.0 res 
--------
[1.0 2.0 0; 0.0 -3.0 6.0; 0.0 0.0 -3.0]
[1.0, -2.0, 0.0]
-0.0
0.6666666666666666
1.0


3-element Vector{Float64}:
 -0.33333333333333326
  0.6666666666666666
  0.0

In [230]:
-1.5-0.25*6

-3.0

In [198]:
P * x

3-element Vector{Float64}:
  4.2
  5.6000000000000005
 21.6

In [12]:
A = [1 2; 3 4]
A

2×2 Matrix{Int64}:
 1  2
 3  4

In [13]:
A[1, 2]

2

In [11]:
zeros(0)

Float64[]

In [6]:
PasovnaMatrika

PasovnaMatrika

In [171]:
P = PasovnaMatrika(Dict([(0,[1, 2, 3]), (-1,[5,6])]), 3)

4.0 l, 
2 ii 1 i 1 mm 1.0
0.0 res 
2 ii 2 i 1 mm 2.0


3×3 PasovnaMatrika{Int64}:
 1  0  0
 5  2  0
 0  6  3

In [173]:
P[1, :]

3-element Vector{Int64}:
 1
 0
 0

In [94]:
P[1,1] = 5

5

In [95]:
a = [1, 2, 3]

3-element Vector{Int64}:
 1
 2
 3

In [100]:
P * a

[5.0, 2.0, 3.0] cc [5.0, 4.0, 9.0]
[5.0, 6.0, 0.0] cc [5.0, 12.0, 0.0]


3-element Vector{Float64}:
  5.0
  9.0
 21.0

In [101]:
P

3×3 PasovnaMatrika{Int64}:
 5  0  0
 5  2  0
 0  6  3

In [102]:
a

3-element Vector{Int64}:
 1
 2
 3